In [1]:
from subprocess import Popen, PIPE
import os
import pathlib
import shutil
import os


In [2]:
#rice
path_mites_dire = 'data/rice/by_family/'
path_out_dir = 'data/rice/by_cluster/'
pid = 0.97
what = 'rice'
min_count = 10
workers = 5

In [3]:
#wheat
path_mites_dire = 'data/wheat/by_family/'
path_out_dir = 'data/wheat/by_cluster/'
pid = 0.97
min_count = 10
workers = 5

In [4]:
files = [f for f in os.listdir(path_mites_dire) if os.path.isfile(os.path.join(path_mites_dire, f))]
count = 0
total = len(files)
for file in files:
    file_path = path_mites_dire + file
    count += 1
    print("%i / %i" % (count,total))
    c_dir = path_out_dir + file + "/"
    if os.path.exists(c_dir):
        continue
    if os.path.isdir(c_dir):
        shutil.rmtree(c_dir)
    pathlib.Path(c_dir).mkdir(parents=True, exist_ok=True)
    cmd_list = [
    './bin/vsearch-2.11.1/bin/vsearch',
    '--cluster_fast', file_path,
    '--threads',str(workers),
    '--strand','both',
    '--clusters', c_dir + "c_",
    '--iddef','1',
    '--id', str(pid)]
    p = Popen(cmd_list, stdout=PIPE, stderr=PIPE)
    out,err = p.communicate()


1 / 6475
2 / 6475
3 / 6475
4 / 6475
5 / 6475
6 / 6475
7 / 6475
8 / 6475
9 / 6475
10 / 6475
11 / 6475
12 / 6475
13 / 6475
14 / 6475
15 / 6475
16 / 6475
17 / 6475
18 / 6475
19 / 6475
20 / 6475
21 / 6475
22 / 6475
23 / 6475
24 / 6475
25 / 6475
26 / 6475
27 / 6475
28 / 6475
29 / 6475
30 / 6475
31 / 6475
32 / 6475
33 / 6475
34 / 6475
35 / 6475
36 / 6475
37 / 6475
38 / 6475
39 / 6475
40 / 6475
41 / 6475
42 / 6475
43 / 6475
44 / 6475
45 / 6475
46 / 6475
47 / 6475
48 / 6475
49 / 6475
50 / 6475
51 / 6475
52 / 6475
53 / 6475
54 / 6475
55 / 6475
56 / 6475
57 / 6475
58 / 6475
59 / 6475
60 / 6475
61 / 6475
62 / 6475
63 / 6475
64 / 6475
65 / 6475
66 / 6475
67 / 6475
68 / 6475
69 / 6475
70 / 6475
71 / 6475
72 / 6475
73 / 6475
74 / 6475
75 / 6475
76 / 6475
77 / 6475
78 / 6475
79 / 6475
80 / 6475
81 / 6475
82 / 6475
83 / 6475
84 / 6475
85 / 6475
86 / 6475
87 / 6475
88 / 6475
89 / 6475
90 / 6475
91 / 6475
92 / 6475
93 / 6475
94 / 6475
95 / 6475
96 / 6475
97 / 6475
98 / 6475
99 / 6475
100 / 6475
101 / 64

In [ ]:
files = [f for f in os.listdir(path_mites_dire) if os.path.isfile(os.path.join(path_mites_dire, f))]
count = 0
total = len(files)
res = {}
for file in files:
    file_path = path_mites_dire + file
    c_dir = path_out_dir + file + "/"
    clusters = [f for f in os.listdir(c_dir) if os.path.isfile(os.path.join(c_dir, f))]
    seqs = []
    for cluster in clusters:
        file_c = open(c_dir + cluster, "r")
        count_seqs = 0
        for line in file_c:
            if line[0:1] == ">":
                count_seqs += 1
        seqs.append(count_seqs)
    if max(seqs) > min_count:
        res[file] = (len(clusters), max(seqs))
        #print(file + ": " + str(len(clusters)) + " max: " + str(max(seqs)))
        #print("*" * 10)
    count += 1
    print("%i / %i" % (count,total))

1 / 6475
2 / 6475
3 / 6475
4 / 6475
5 / 6475
6 / 6475
7 / 6475
8 / 6475
9 / 6475
10 / 6475
11 / 6475
12 / 6475
13 / 6475
14 / 6475
15 / 6475
16 / 6475
17 / 6475
18 / 6475
19 / 6475
20 / 6475
21 / 6475
22 / 6475
23 / 6475
24 / 6475
25 / 6475
26 / 6475
27 / 6475
28 / 6475
29 / 6475
30 / 6475
31 / 6475
32 / 6475
33 / 6475
34 / 6475
35 / 6475
36 / 6475
37 / 6475
38 / 6475
39 / 6475
40 / 6475
41 / 6475
42 / 6475
43 / 6475
44 / 6475
45 / 6475
46 / 6475
47 / 6475
48 / 6475
49 / 6475
50 / 6475
51 / 6475
52 / 6475
53 / 6475
54 / 6475
55 / 6475
56 / 6475
57 / 6475
58 / 6475
59 / 6475
60 / 6475
61 / 6475
62 / 6475
63 / 6475
64 / 6475
65 / 6475
66 / 6475
67 / 6475
68 / 6475
69 / 6475
70 / 6475
71 / 6475
72 / 6475
73 / 6475
74 / 6475
75 / 6475
76 / 6475
77 / 6475
78 / 6475
79 / 6475
80 / 6475
81 / 6475
82 / 6475
83 / 6475
84 / 6475
85 / 6475
86 / 6475
87 / 6475
88 / 6475
89 / 6475
90 / 6475
91 / 6475
92 / 6475
93 / 6475
94 / 6475
95 / 6475
96 / 6475
97 / 6475
98 / 6475
99 / 6475
100 / 6475
101 / 64

In [ ]:
res_s = sorted(res.items(), key=lambda x:x[1][0])

In [ ]:
#res_s

In [ ]:
with open('data/' + what + '/similar_mites_count.csv', 'w') as f:
    for v in res_s:
        f.write("%s,%s,%s\n"%(v[0],v[1][0],v[1][1]))
